In [1]:
import tensorflow as tf
import numpy as np

In [2]:
#Artificial dataset, lets say XOR
def xor():
    X = np.array([[0,1], [1,0], [1,1],[0,0]], dtype=float)
    Y = np.array([1,1,0,0], dtype=float).reshape((-1,1))
    return X,Y

def square_function():
    X = []
    Y = []
    for x in range(10):
        X.append(x)
        Y.append(x**2)
    return np.array(X,dtype=np.float32).reshape(-1,1),np.array(Y, dtype=np.float32).reshape(-1,1)

def high_dim():
    X = []
    Y = []
    for x in range(100):
        X.append([x, x+3, x+4, x+5, x+10, x+20, x+60])
    return np.array(X,dtype=np.float32).reshape(-1,7), None



# Simple autoencoder

So basically the only thing that is changing is the loss function, we want to get the same output as the input. This can be used to reduce the dimensionality of the input (encoding) with hidden layers, then we get the input back by decoding later, this is what the network is training.

In [7]:
#input

X,Y = high_dim()

batch_size = 10

#Helper functions
def fully_connected(x, size, activation=tf.nn.relu):
    b = tf.Variable(tf.zeros([size]))
    weights = tf.Variable(tf.truncated_normal([x.get_shape()[1].value, size]))
    return activation(b + tf.matmul(x, weights))


INPUT_SHAPE = X.shape

x = tf.placeholder("float", [None,X.shape[1]])

#simple neural network to solve the xor problem

h1 = fully_connected(x, 20, activation=tf.nn.tanh)
print(h1.get_shape())

h2 = fully_connected(h1, 20)
print(h2.get_shape())

h3 = fully_connected(h2, 4)
print(h3.get_shape())

h4 = fully_connected(h3, 20)
print(h4.get_shape())

out = fully_connected(tf.nn.dropout(h4, 0.5), X.shape[1])
print(out.get_shape())

loss = tf.reduce_mean((out-x)**2)
tf.scalar_summary("loss", loss)


optimizer = tf.train.AdamOptimizer(0.1)
train_step = optimizer.minimize(loss)



(?, 20)
(?, 20)
(?, 4)
(?, 20)
(?, 7)


# Run the session

In [10]:
init =  tf.initialize_all_variables()

num_epochs=10000
#merge = tf.merge_all_summaries()

with tf.Session() as sess:
    
    writer = tf.train.SummaryWriter("./summary", sess.graph)
    sess.run(init)
    #sess.run(merge)

    for epoch in range(num_epochs):
        for batch in range(X.shape[0]//batch_size):
            feed_dict = {
                x : X[batch*batch_size:batch*batch_size + batch_size],
            }
            _, lossVal = sess.run([train_step, loss], feed_dict=feed_dict)
        #writer.add_summary(merged, epoch)
        if(epoch%40 == 0):
            print("%d. Epoch Loss: %.4f "  %(epoch, lossVal))
        
        

0. Epoch Loss: 6731.2368 
40. Epoch Loss: 4527.8496 
80. Epoch Loss: 4075.4465 
120. Epoch Loss: 4126.0259 
160. Epoch Loss: 3960.4563 
200. Epoch Loss: 3520.7549 
240. Epoch Loss: 4118.2285 
280. Epoch Loss: 3541.5635 
320. Epoch Loss: 3261.5769 
360. Epoch Loss: 3412.9380 
400. Epoch Loss: 3228.7542 
440. Epoch Loss: 3270.4038 
480. Epoch Loss: 3346.4084 
520. Epoch Loss: 3223.3364 
560. Epoch Loss: 3267.9849 
600. Epoch Loss: 3192.3806 
640. Epoch Loss: 3234.2644 
680. Epoch Loss: 3198.5796 
720. Epoch Loss: 3203.4055 
760. Epoch Loss: 3174.9805 
800. Epoch Loss: 3164.3376 
840. Epoch Loss: 3160.4604 
880. Epoch Loss: 3159.6013 
920. Epoch Loss: 3159.1167 
960. Epoch Loss: 3159.0447 
1000. Epoch Loss: 3159.0198 
1040. Epoch Loss: 3159.0156 
1080. Epoch Loss: 3159.0139 
1120. Epoch Loss: 3159.0149 
1160. Epoch Loss: 3159.0134 
1200. Epoch Loss: 3159.0142 
1240. Epoch Loss: 3159.0193 
1280. Epoch Loss: 3159.0186 
1320. Epoch Loss: 3159.0178 
1360. Epoch Loss: 3159.0173 
1400. Epoch Lo

KeyboardInterrupt: 